In [1]:
##This program to read data from PLC mirco 800- AB, creare log, update data in the cloud, send SMS and Emails in case of alarms

#import timeit
#from datetime import datetime
#from tags import *
#import matplotlib.pyplot as plt
#from lgx_gui_test import *
#from flask import Flask, render_template, request
# import os.path
# import quandl
# import csv
#from generic_messaging import *

from basic_reads import *
from basic_writes import *
from send_sms import *
from send_email import *
import time
import pandas as pd
import gspread

In [2]:
#_____open API for google sheet to update the data log
try:

    gc = gspread.service_account(filename='fast-feed-alerts.json')
    sh=gc.open('fast-feed-alert')
    shProfile = sh.get_worksheet(1)
    shData_log = sh.get_worksheet(0)
    shData_log.append_row(['JETSON', 'NANO',  'CONNECTED TO', 'FASTFEED', ' Micro 800 - AB'])
    print('succesfully established connection with JETNANO and google spread sheet')
except Exception as e:
    print('Failed to open connection with gspread API, Error: ', str(e))


succesfully established connection with JETNANO and google spread sheet


# Send Data to Google sheets, Send message, and Email


In [3]:
#___________________Send SMS____________
#___________________Send Email____________
#___________________Send Data to google Sheet
#___________________Send Null Data to Google Sheet in case of lost connection_____________

df = pd.DataFrame()
start = time.time()

send_message = False
low_to_high =  True
high_to_low =  False


print('__________________Program_Start______________________')

try:
    shData_log.append_row(['Time' ,'_IO_EM_DI_00: Button', '_IO_EM_DI_01: Capactive', '_IO_EM_DI_02: Inductive', '_IO_EM_DI_03', '_IO_EM_AI_00: Pressure', '_IO_EM_AI_01: Key', '_IO_P3_AO_00','_IO_P3_AO_01' ,'_IO_EM_DO_08 Lamp' ])
    print('___________open connection with google sheet API and updated row index_______________')
except Exception as e:
    print('Failed to open connection to gspread API, Error: ', str(e))



#while i<2:                   # Activated if we want to run the program forever
x = 60*60*2                   # Activated if we want to run the program for specific duration (seconds * minutes *hours)
for i in range(5):                              # we can just specifiy how many datalog we want
    #time.sleep(.7)
    seconds = time.time()
    local_time = time.ctime(seconds)
    
    #__________print datalog locally___________
    try:
        print(local_time,read_single0().value, read_single1().value, read_single2().value, read_single3().value,
              read_singleAI0().value, read_singleAI1().value, read_singleAO0().value, read_singleAO1().value, 
              read_singleO8().value)
    except Exception as e:
        print(' cannot print Log, Error: ', str(e))
        
    
    #_________update the cloud_google spread sheet_______________
    try:
        shData_log.append_row([local_time ,read_single0().value, read_single1().value, read_single2().value, 
                               read_single3().value, read_singleAI0().value, read_singleAI1().value, read_singleAO0().value,
                               read_singleAO1().value,read_singleO8().value])
        
        print('_______________Cloud_succefully_updated_____________')
    except Exception as e:
        try:
            shData_log.append_row([local_time ,'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null'])
        except Exception as e1:
            print("Failed to send data to the cloud with NULL, Error: ", str(e1))
        print("Failed to send data to google sheet; Error: ", str(e))
        #time.sleep(5)
        
        
    #__________sending an alarm by email and/or message________________
    try:        
        if read_singleAI0().value > 50000 and send_message and low_to_high:
            message = 'You have exceeded the high-level limit. Take an immediate action please, current reading of the valve position is:    ' + str(read_singleAI0().value)
            print(message)
            send_sms(message)
            send_email(message)

            low_to_high =  False
            high_to_low = True
    except Exception as e:
        print('Alert SMS for high level limit cannot be sent, Error: ', str(e))
            
    #__________sending a confirmation message that everyhing is okay by email and/or message________________   
    try:
        if read_singleAI0().value < 30000 and send_message and  high_to_low:
            message = 'You are all set, Current Reading for valve position:    ' + str(read_singleAI0().value)
            print(message)
            send_sms(message)
            send_email(message)

            low_to_high = True
            high_to_low = False  
    except Exception as e:
        print('Alert SMS for low level limit cannot be sent, Error: ', str(e))
        
    #___________________Save the log to DataFrame____________________________        
    try:
        
        # print(local_time,read_single0().value, read_single1().value, read_single2().value, read_single3().value,
        #       read_singleAI0().value, read_singleAI1().value, read_singleAO0().value, read_singleAO1().value, 
        #       read_singleO8().value)
        
        df1 = pd.DataFrame(data=(read_single0().value, read_single1().value, read_single2().value, read_single3().value,
                                 read_singleAI0().value, read_singleAI1().value, read_singleAO0().value, 
                                 read_singleAO1().value,read_singleO8().value), columns= {local_time}, 
                           index = ['_IO_EM_DI_00: Button', '_IO_EM_DI_01: Capactive', '_IO_EM_DI_02: Inductive',
                                    '_IO_EM_DI_03', '_IO_EM_AI_00: Pressure', '_IO_EM_AI_01: Key', '_IO_P3_AO_00',
                                    '_IO_P3_AO_01' ,'_IO_EM_DO_08 Lamp' ]).T

       
        df = pd.concat([df, df1])
        print('_______________DataFrame_updated_____________')
        
    except Exception as e:
        try:
            df1 = pd.DataFrame(data=('Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null','Null','Null'),
                           columns= {local_time}, index = ['_IO_EM_DI_00: Button', '_IO_EM_DI_01: Capactive', 
                                                           '_IO_EM_DI_02: Inductive', '_IO_EM_DI_03', 
                                                           '_IO_EM_AI_00: Pressure', '_IO_EM_AI_01: Key', 
                                                           '_IO_P3_AO_00','_IO_P3_AO_01' ,'_IO_EM_DO_08 Lamp' ]).T
        except Exception as e:
            print("Failed to update dataframe with NULL", str(e))
                                                                  
        print('DataFrame_log cannot be updated, Error: ', str(e))
    
    
    

end = time.time()

print('Program time taken:  ', end - start)

print(local_time)
x = str(local_time.replace(":","-"))
name_of_file = x.replace(" ", "_")

# #print(df)
# PATH = r'Datalog_' + name_of_file + '.csv' #(use "r" before the path string to address special character, such as '\'). Don't forget to put the file name at the end of the path + '.csv'
# df.to_csv(PATH)


#Save to windows
PATH1 = r'C:\ML_Projects\ABB_Pycomm_ver_2\examples\Datasets\Datalog_' + name_of_file + '.csv' #(use "r" before the path string to address special character, such as '\'). Don't forget to put the file name at the end of the path + '.csv'

print('PATH1', PATH1)
try:
    df.to_csv(PATH1)
except Exception as e:
    print('Failed to save datalog to CSV file, Error: ', str(e))








__________________Program_Start______________________
___________open connection with google sheet API and updated row index_______________
 cannot print Log, Error:  failed to open a connection
Failed to send data to google sheet; Error: failed to open a connection
Alert SMS for high level limit cannot be sent, Error: failed to open a connection
Alert SMS for low level limit cannot be sent, Error: failed to open a connection
DataFrame_log cannot be updated,updated with Null instead, Error: failed to open a connection
 cannot print Log, Error:  failed to open a connection
Failed to send data to google sheet; Error: failed to open a connection
Alert SMS for high level limit cannot be sent, Error: failed to open a connection
Alert SMS for low level limit cannot be sent, Error: failed to open a connection
DataFrame_log cannot be updated,updated with Null instead, Error: failed to open a connection
 cannot print Log, Error:  failed to open a connection
Failed to send data to google sheet; 

In [4]:
#___________read muliple tags at once_________
print('read_multiple tags at once: \n' ,read_multiple())

CommError: failed to open a connection

### Save to LINUX

In [ ]:
#Run these lines to save files @ Linux Environments

#PATH2 = r'/mnt/c/ML_Projects/ABB_Pycomm/docker_examples_ver2/Datalog_Unix_' + name_of_file + '.csv' #(use "r" before the path string to address special character, such as '\'). Don't forget to put the file name at the end of the path + '.csv'
#save_path = r'/mnt/c/ML_Projects/ABB_Pycomm_ver_2/examples'

#PATH2 = os.path.join(save_path, name_of_file, ".csv")
# PATH2 = os.path.join(save_path, "name_of_file.csv")

#print('PATH2' ,PATH2)
#df.to_csv(PATH2)


PATH2 C:\ML_Projects\ABB_Pycomm\examples\Datasets\Fri_Feb_18_10-51-15_2022\.csv


## Write Value to Controller

In [ ]:
# Write tags to the controller
print('___________Start writing_____________')
try:
    write_single_setA(7)
except Exception as e:
    print("Failed to update Position_SetpointA ,ERROR: " + str(e))

try:
    write_single_setB(9)
except Exception as e:
    print("Failed to update Position_SetpointA ,ERROR: " + str(e))

___________Start writing_____________
write_single_Position_SetpointA: ..............start writing.................
write_single_Position_SetpointB: ..............start writing.................
